In [35]:
import pandas as pd
import json


# Generating a dataframe where each column is a business and each row is an individual review

In [36]:
df_filtered = pd.read_csv('../data/prepped_data_n_5_r.csv')

with open("../data/yelp_dataset/yelp_academic_dataset_review.json", "r", encoding="utf-8") as file:
    data = [json.loads (line) for line in file]

# Convert the list of dictionaries into a DataFrame
df_reviews = pd.DataFrame(data)

df_reviews

In [37]:
# filter df_reviews to only include business id's in df_filtered

df_reviews_filtered = df_reviews[df_reviews['business_id'].isin(df_filtered['business_id'])]

df_reviews_filtered = df_reviews_filtered.copy()

# Create a new index within the original DataFrame
df_reviews_filtered.loc[:, 'review_index'] = df_reviews_filtered.groupby('business_id').cumcount()

# Pivot the DataFrame
df_pivoted = df_reviews_filtered.pivot(index='review_index', columns='business_id', values='text')

df_pivoted.to_csv('../data/business_reviews_nlp.csv')


In [38]:
df_reviews = pd.read_csv('../data/business_reviews_nlp.csv', index_col='review_index')

pd.set_option('display.max_colwidth', 40)

df_reviews

/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/ipykernel_37127/2902560824.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

,--onnLZrsCazmcy2P_7fcw,-3AooxIkg38UyUdlz5oXdw,-8iATYRnN46Km0_-ldx6cg,-9r8nAzWyRSLxBWt8uQOdA,-ALqLSTzkGDMscHdxA1NgA,-BdYhP-12elmFV7oB1iv4A,-FM4CxOg4XXmX_Ebky_SiQ,-FRHYI6doGCjIsXiKHkUqw,-Fka99c-tJ-epWYNIobqyQ,-H-fZonTLitwHFY_PLAIjA,...,zVxDd79gdZcfz-O14QBCrA,zWwGjQlUEiLS7hCXDkBGzg,zYTQOpgMSMaFNkcCAvJR3Q,zZrDoiQIUmiVkifJx0h_KA,zbrIMldF_O1ZQ0vpUaaa8A,zeAfmYy9b1gfUJLRBHm6vQ,ziXR7sUrbKRCNeDqfjibUg,zpuFEeAhrNzXPkOlBf5Kog,zu4p6IZLSVn2Noto-vcwzw,zxW8zECvT_SqejieMMjb5A
review_index,,,,,,,,,,,,,,,,,,,,,
0,Another place where I'm a regular wh...,Wow. Wow. We've been trying to go he...,Didn't even get to eat here since th...,Let the Vegan Hunger Games begin and...,Excellent fish taco. Cold corona. Sy...,New favorite neighborhood spot! Staf...,The wife and I had dinner this eveni...,college years review:\n\nwe lived a ...,I ordered delivery online through Ye...,i used to work there like 2 yrs ago....,...,bomb.\n\nsanta barbara burger with e...,Their zoso gyro is delicious and I'd...,Arturo's is my one and only batting ...,I have gone to this location on thre...,Jasmine our Waitress did a great job...,chicken picattaaaaaaaaaa.,4 stars?\n\nChick behind the counter...,"I've always loved outback, but the s...","Everytime I'm in Santa Barbara, this...","Being from SoCal, I know the differe..."
1,This sizzler had bugs in the food. A...,There are not enough superlatives! A...,See all the five-star reviews? They'...,The food is alright. I honestly expe...,The food was ok but not as good as s...,Corner Tap is a welcome addition to ...,Went there for a birthday brunch. F...,I've been here three times. I was im...,"I know I know, silly as it seems, I ...",Good pizza. Giant salads. Awesome ...,...,I'm a sucker for great service and t...,This place is amazing! They pack the...,The only reason I didn't give Arturo...,So excited to have my favorite coffe...,My family chose Bluewater Grill to c...,Simply delicious! We ate here two da...,I got the chicken burrito to go. It ...,I had the baby back ribs and spring ...,Had the pleasure of eating breakfast...,How does this place have a 4-star av...
2,"I worked at this sizzler in 1994, 19...",Horrible service. Our waiter never c...,GREAT burger. Some burgers are packe...,"Trash service, took almost half an h...",Very authentic mexican food. The bir...,I still haven't been inside to sit d...,So I hve to say a bit pricey (of cou...,@ least once a week I came here to f...,I used to love Pizza Mizza. Somethin...,The new owner has done a great job w...,...,Wellllll......... I came here for lu...,Greek house blend gyro with salad is...,"4 stars food, service generally good...",They recently opened a new location....,Food and atmosphere was very good. ...,This place is great if you want an a...,Chose this place after seeing someon...,This is the best Outback Steakhouse ...,Lovely patio to people watch and the...,My wife and I were visiting SB and l...
3,Yummy. I've eaten here twice in the...,We stopped in for a quick bite for l...,"Good place, food was great. A little...",Really enjoy their vegan option as I...,Good overall taste to the food but c...,This is a great little lunch spot. T...,One of the best duck dishes I've eve...,I tried the hot chicken wings today....,Delish! One of my favorite sb pizza ...,Omg! I'm totally impressed! Haven't ...,...,This place just opened on Monday 7/1...,The food here was pretty good. The ...,This was a fun adventure down Milpas...,What a fantastic place to come and s...,We were really looking forward to wh...,Really good food big portions\nI usu...,NASTY BATHROOMS! covered with graffi...,Went in for an early dinner. Ordere...,Chronicles of Cali BOB (Article #2)....,This review is solely based on one t...
4,What the hell happened to this place...,Enjoyed a great dinner at the bar on...,PLEASE READ ENTIRE REVIEW\n\nAs a cu...,Went here the other day for the firs...,Sorry to disagree with higher marks ...,Been living in Santa Barbara for 8 y...,Call at 5:59pm asking for a 7pm rese...,

# setting atmosphere keywords. 

In [39]:

atmosphere_keywords = [
    "romantic", "intimate", "birthday", "anniversary",
     "friends", "unique", "celebration", 
    "live music","outdoor seating","upscale", "rooftop", "waterfront", 
    "scenic", "hidden gem", "cozy", "affordable", "fine-dining", "dimly lit", "aesthetic","casual"
]


# creating dictionaries to map business id's with filtered reviews containing one of the keywords and to map keywords to a list of restaurants

In [40]:
# Assuming each column is a restaurant (business_id) and each row is a review
ambiance_reviews_dict = {}

for column in df_reviews.columns:
    # Filter reviews in each column
    filtered_reviews = df_reviews[column].apply(lambda review: 
                                                any(keyword in str(review).lower() 
                                                    for keyword in atmosphere_keywords))
    # Get the list of filtered reviews for the current column (business_id)
    filtered_reviews_list = df_reviews[column][filtered_reviews].tolist()
    ambiance_reviews_dict[column] = filtered_reviews_list

# Now ambiance_reviews_dict is a dictionary with business_ids as keys and lists of filtered reviews as values



# Initialize an empty dictionary
keyword_to_restaurant_ids = {keyword: [] for keyword in atmosphere_keywords}

# Iterate through each column (restaurant ID)
for restaurant_id in df_reviews.columns:
    # Get all reviews for the current restaurant
    reviews = df_reviews[restaurant_id].tolist()
    
    # Check each keyword in the current column's reviews
    for keyword in atmosphere_keywords:
        # If keyword is found in any review, add the restaurant ID to the keyword's list
        if any(keyword in str(review).lower() for review in reviews):
            keyword_to_restaurant_ids[keyword].append(restaurant_id)

# Optionally, remove duplicates if needed
for keyword in keyword_to_restaurant_ids:
    keyword_to_restaurant_ids[keyword] = list(set(keyword_to_restaurant_ids[keyword]))



keyword_to_restaurant_ids



{'romantic': ['CoZ2mpsMBP8HUG1ymKoZTg',
  'RNzhLU8zy1L5dBkjRK4xEA',
  'iHdrLu8deF5GceB0V1rPhA',
  'ERzIS-WRLw-80_B4yqcG3Q',
  'jMAQupx06hHjie54wV2yNQ',
  'VAIOtbsxdNmuhUKkQZFRlA',
  '0it-2gWk_gyuZyEMQzBBuQ',
  'vi5xjfEhV6rnXrC_GbdhnA',
  'eX7o_-s5TmDT-DMfTV4cmw',
  'PZIVcSuN-G_I6oG6EVKntQ',
  'CLlnfxaYPTVLKE4VR-U1Yg',
  'KSYONgGtrK0nKXfroB-bwg',
  'ZKi4F9BlA46rLvLyeIE3VA',
  'yPSejq3_erxo9zdVYTBnZA',
  'CkENBnSJFfPA1hY0qrqfCg',
  'kbWKwvLXfJm5sLVfw-Foww',
  'noiUCN8A9Vdm9pUbWX6K-w',
  'n0Z10UAJO187H8Zp8gl-rw',
  '4IqrY1mXK65HgrrgkpE-cw',
  '3xJ_6qxRA0sdUm3ThCPiCw',
  'XkZwD46WcsAlbpg1inwbLQ',
  'ctVn7wMvJX8f4Z9SnOnrHw',
  '1J0NIfmrSuV72KzhmYFxMA',
  'R6NxHoRRK5Y4ishGsLrGwA',
  'fqStx0SvwMFChX_HIdUeEQ',
  'IoAubn7CpU2rqcURxUo1MA',
  '8_r9EZL6vX5fPBoVh3gUrA',
  'xGaYXbXGwagUKPNEaVqbWw',
  'eHRzuUXA2jrR99ecioWU4g',
  '18eWJFJbXyR9j_5xfcRLYA',
  'BErYojCJSO959vzaz9JclQ',
  'hPzPpfSjgQkpWCD7YjcY-A',
  'Hl_vXTiYLozWo7T9Cq17vw',
  'IZZ7J14VnJGyUqYH1tgT6w',
  '-TArOdux8Z4oH-jIM1FJuQ',
  'E7RtI

In [41]:

# Write the dictionary to a JSON file. This file is critical for the hybrid recc system. 
with open('../data/keywords_restaurants.json', 'w') as file:
    json.dump(keyword_to_restaurant_ids, file, indent=4)

# checking to see how many restaurants were excluded

In [42]:
# Creating a new dictionary with only restaurants that have no reviews
restaurants_with_no_reviews = {key: value for key, value in ambiance_reviews_dict.items() if len(value) == 0}

print(len(restaurants_with_no_reviews))


print(len(ambiance_reviews_dict))

90
1153


# checking to see if any keywords have no associated restaurants 

In [43]:
# Check for keys with empty values
for keyword, restaurant_ids in keyword_to_restaurant_ids.items():
    if not restaurant_ids:  # This checks for an empty list
        print(f"The key '{keyword}' has an empty value.")

# Use this section to look at filtered reviews of specific restuarants 

In [44]:
ambiance_reviews_dict['sk0IM8koYo1o9-k_rpkmxQ']

['Amazing Tacos! Carne Asada, Al Pastor, and the veggie tacos are my favorites. Definitely recommend coming here for the salsas as well! They have their own delicious unique flavors! Yum yum!!',
 'Great food and indoor and outdoor seating. The prices are good and the staff is always friendly.',
 'This is actually my second time eating at this wonderful Mexican restaurant. First, it is incredibly clean inside. They have all the classics ,tacos ,burritos, quesadillas ,tortas.... But what makes this place truly exceptional is a huge amount of different types of meats you can \nchoose from. Everything from lengua to al pastor, shredded beef and the list goes on.... They have outdoor seating and four or five different types of salsa. The prices are reasonable and the servings are quite large. Personally I highly recommend the Torta Cubana.',
 "Decided we wanted tacos for lunch while on State Street. So glad we stopped here. The tortillas are home made which is a rarity nowadays. They really